# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [40]:
sc

Calculation started (calculation_id=68c5bec0-0742-f8be-fef4-b523bd59ed01) in (session=54c5bea8-91f5-a79f-c1e5-9eea4217c3a5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [41]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=aec5bec0-0c7c-43fa-0482-0acd31fbeb8d) in (session=54c5bea8-91f5-a79f-c1e5-9eea4217c3a5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [42]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=0ac5bec0-0dae-ef16-de13-7495732fbf84) in (session=54c5bea8-91f5-a79f-c1e5-9eea4217c3a5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [43]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'yyang3041'

Calculation started (calculation_id=dec5bec0-0f12-6e9e-cc99-57a379e60cc1) in (session=54c5bea8-91f5-a79f-c1e5-9eea4217c3a5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [44]:
#export
def long_trips(trips):
    ret = trips.withColumn("trip_distance", trips["trip_distance"].cast("double"))
    ret = trips.filter(col("trip_distance") >= 2)
    return ret

Calculation started (calculation_id=6cc5bec0-1082-852e-888e-818d1081caeb) in (session=54c5bea8-91f5-a79f-c1e5-9eea4217c3a5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [45]:
#export
def manhattan_trips(trips, lookup):
    ret = lookup.join(trips, lookup["LocationID"]==trips["DOLocationID"], "left").filter(col("Borough")=="Manhattan")
    ret = ret.withColumn("pc", col("passenger_count").cast("int"))
    ret = ret.groupBy(col("DOLocationID")).agg({"pc":"sum"})
    ret = ret.withColumnRenamed("sum(pc)", "pcount").orderBy(col("pcount").desc())
    ret = ret.select(col("DOLocationID"), col("pcount")).limit(20)
    return ret

Calculation started (calculation_id=c0c5bec0-11b8-281c-f24f-a98402c72e6b) in (session=54c5bea8-91f5-a79f-c1e5-9eea4217c3a5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [46]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    df1 = trips.withColumn("ta", col("total_amount").cast("double"))
    df1 = df1.groupBy(trips["PULocationID"]).agg({"ta":"avg", "PULocationID":"count"})
    df_joined = trips.join(mtrips, trips["DOLocationID"] == mtrips["DOLocationID"], "right")
    df_joined = df_joined.groupBy("PULocationID").agg({"PULocationID":"count"})
    df = df1.join(df_joined, df1["PULocationID"]==df_joined["PULocationID"], "right")
    df = df.withColumn("weighted_profit", df_joined["count(PULocationID)"].cast("double")/df1["count(PULocationID)"].cast("double")*col("avg(ta)"))
    df = df.select(df1["PULocationID"], col("weighted_profit"))
    df = df.withColumn("weighted_profit", col("weighted_profit").cast("double"))
    return df

Calculation started (calculation_id=82c5bec0-12f6-8d38-fa38-c2654cd75404) in (session=54c5bea8-91f5-a79f-c1e5-9eea4217c3a5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [47]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    df = wp.join(lookup, wp["PULocationID"]==lookup['LocationID'], "left").orderBy(col("weighted_profit").desc()).limit(20)
    df = df.select(col("Zone"), col("Borough"), col("weighted_profit"))
    return df

Calculation started (calculation_id=1ec5bec0-183e-be2f-1392-aceba63e8d98) in (session=54c5bea8-91f5-a79f-c1e5-9eea4217c3a5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [48]:
#main("large", "s3://cse6242-yyang3041/output-large_final")

Calculation started (calculation_id=38c5bec0-1972-76be-0dc4-c9d8b0dbf00f) in (session=54c5bea8-91f5-a79f-c1e5-9eea4217c3a5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
yyang3041
Trip Count:  187203269
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens| 29.36045577913085|
|Flushing Meadows-...|       Queens| 27.30484573361766|
|       South Jamaica|       Queens|26.294916239873483|
|     Randalls Island|    Manhattan| 24.15098994022752|
|        Astoria Park|       Queens| 21.70641711214752|
|Briarwood/Jamaica...|       Queens|19.945064631789332|
|Springfield Garde...|       Queens|19.468309288781906|
|             Jamaica|       Queens|  19.2839430001379|
|              Corona|       Queens|18.228769248155967|
|   LaGuardia Airport|       Queens|18.181338808373003|
|         Jamaica Bay|       Queens|17.100529446757893|
|             Maspeth|       Queens| 17.00545064007954|
|Eltingville/Annad...|Staten Island|16.837764756944445|
|         JFK Airport|       Queens|16.777725348